In [ ]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
from src.neural_network import NeuralNetwork
from src.optimizers import Adam
from src.trainer import Trainer
from src.utils import train_val_test_split, one_hot_encode

# Cargar Iris
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']
df = pd.read_csv(url, names=cols)

# Mapear clases
class_map = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
df['class'] = df['class'].map(class_map)

X = df.iloc[:, :-1].values
y = df['class'].values

# Normalización
X = (X - X.mean(axis=0)) / X.std(axis=0)

# One-hot
y_oh = one_hot_encode(y, num_classes=3)

# Split
X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(
    X, y_oh, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, 
    random_seed=42
)

# Red: 4 -> 8 -> 3
architecture = [
    (4, None),
    (8, 'sigmoid'),
    (3, 'softmax')
]

model = NeuralNetwork(architecture)
optimizer = Adam(learning_rate=0.01)
trainer = Trainer(model, optimizer)

# Entrenar con minibatches
trainer.train(X_train, y_train, X_val, y_val, 
              epochs=500, batch_size=16, verbose=True)

# Test
test_loss, test_acc = trainer.evaluate(X_test, y_test)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

trainer.plot_history()